In [23]:
import numpy as np
import pandas as pd
import sys
from langdetect import detect_langs

In [14]:
# get most important coeffs from lasso-logit-full
coeffs = np.loadtxt("../coef_lasso_logit_full.txt")
top_words = coeffs.argsort()[-10:][::-1]
print(coeffs[top_words])

[2.88317862 2.20474637 1.89043858 1.88126299 1.84982683 1.77917599
 1.74436485 1.67609757 1.64098876 1.55399143]


In [15]:
# look at which words correspond to the most important coeff
vocablist = pd.read_csv("../vocab10K.csv")
word_map = np.loadtxt("../i_keep_columns.txt")
vocab_indices = word_map[top_words] + 1
print(vocablist.loc[vocablist['index'].isin(vocab_indices)])

      index        word  female  i_pronoun  exclude      coef        ME  \
1106   1107         hot     NaN        NaN      NaN  1.849972  0.271002   
1530   1531   beautiful     NaN        NaN      NaN  1.637034  0.239809   
1696   1697  attractive     NaN        NaN      NaN  1.672941  0.245069   
3298   3299       marry     NaN        NaN      NaN  1.879690  0.275356   
4135   4136    pregnant     NaN        NaN      NaN  2.203002  0.322718   
5067   5068   pregnancy     NaN        NaN      NaN  1.734884  0.254143   
7146   7147    marrying     NaN        NaN      NaN  1.775510  0.260094   
7889   7890      breast     NaN        NaN      NaN  1.546361  0.226526   
8914   8915      hotter     NaN        NaN      NaN  2.882882  0.422313   
9137   9138        plow     NaN        NaN      NaN  1.892775  0.277273   

      nFemale  nMale  coef_pronoun  ME_pronoun  nFemale_pronoun  \
1106     3613   1053      1.220669    0.197421             1309   
1530     1419    610      0.925590    0.

In [16]:
# HELPER: dump all the gender coded words into a file
vocab10K=pd.read_csv("../vocab10K.csv")
male = vocab10K.loc[vocab10K['female'] == 0,:]
male = male['word'].tolist()
female = vocab10K.loc[vocab10K['female'] == 1,:]
female = female['word'].tolist()

In [32]:
# HELPER: label data
#path = "../custom_data/all_tweets.csv"
path = "../custom_data/economics_posts.csv"
data = pd.read_csv(path)
vocab = pd.read_csv("../vocab10K.csv")
num_entries = data.shape[0]
y = list()
male_set = set(male)
female_set = set(female)
nFemale = 0
nMale = 0
unambiguous_posts = list()
for i in range(num_entries):
    if i % 1000 == 0:
        print(i)
    post = str(data['text'][i]).lower()
    lang = "en"
    try:
        lang = str(detect_langs(post))
    except Exception as e:
        pass
    if "en" not in lang:
        continue
    i_male, i_female = 0, 0
    for word in post.split():
        if word in male_set:
            i_male += 1
        elif word in female_set:
            i_female += 1
    if i_male > 0 and i_female == 0:
        y.append(0)
        nMale += 1
        unambiguous_posts.append(post)
    if i_female > 0 and i_male == 0:
        y.append(1)
        nFemale += 1
        unambiguous_posts.append(post)
print("nMale: ", nMale)
print("nFemale: ", nFemale)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
nMale:  2135
nFemale:  299


In [33]:
# Helper: make bow
vocab = pd.read_csv("../vocab10K.csv")
num_entries = len(unambiguous_posts)
X = np.zeros((num_entries, 10000))
vocab_list = vocab['word'].tolist()
vocab_set = set(vocab_list)
for i in range(num_entries):
    if i % 100 == 0:
        print(i)
    post = unambiguous_posts[i].split()
    for word in post:
        if word in vocab_set:
            X[i, vocab_list.index(word)] = 1

np.save("reddit_data", X)
np.save("reddit_labels", y)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400


In [34]:
# get most important coeffs from lasso-reddit
coeffs = np.loadtxt("../coef_reddit.txt")
women_words = coeffs.argsort()[-15:][::-1]
men_words = coeffs.argsort()[:15]
# look at which words correspond to the most important coeff
word_map = np.loadtxt("../i_keep_columns.txt")
women_indices = word_map[women_words] + 1
men_indices = word_map[men_words] + 1
print("women: ")
print(vocablist.loc[vocablist['index'].isin(women_indices)]['word'])
print("men: ")
print(vocablist.loc[vocablist['index'].isin(men_indices)]['word'])

women: 
6          and
18          it
53          my
109        had
113       want
145         am
200       both
206       look
281    student
414       home
462       kids
529     living
586      spend
614       debt
665       stay
Name: word, dtype: object
men: 
14         that
32         this
46         will
72        which
148         job
160        same
187        back
203    economic
238        give
258         end
286       using
328       large
367      system
796     company
841      worked
Name: word, dtype: object


In [9]:
# count instances of word "Yellen" (case-insensitive)
import pandas as pd
vocab = pd.read_csv("../gendered_posts.csv")
list_of_posts = vocab['raw_post'].tolist()
y_count = 0
for post in list_of_posts:
    p = post.lower()
    if "yellen" in p:
        y_count += 1
print(y_count)

464
